In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch 
import torch.nn as nn
import torch.nn.functional as F
import time
import math
# 数据预处理相关函数
from Reader import Footscan_reader
from Reader import gait_data_reader
import random
import pickle
import os
from torch.utils.data import DataLoader,SubsetRandomSampler
LEARING_RATE=0.001
BATCH_SIZE=128
DEVICE=torch.device('cuda')

footscan_reader = Footscan_reader()
gaitdata_reader=gait_data_reader()
torch.cuda.set_device(1)

## 1.read the data


In [5]:
info_dic=pickle.load(open('info_dic.pkl','rb'))
output_info=pickle.load(open('../../processed_data/final_output.pkl','rb'))
input_array=np.load('../../processed_data/final_input.npy')
# input_array=np.expand_dims(input_array,2)

print('load all data from data buffer file')
print('\n','*'*50,'\n')

valid_input=[]
valid_norm_moment=[]
valid_delta=[]
valid_task=[]
for i in range(input_array.shape[0]):
    valid_input.append(input_array[i,:,:,:])
    valid_norm_moment.append(output_info[i].norm_data)
    valid_delta.append(output_info[i].delta)
    valid_task.append(output_info[i].group)

# remove a bad data with 80k loss
index_great_loss=397
if len(output_info)==453:
    del valid_input[index_great_loss]
    del valid_delta[index_great_loss]
    del valid_norm_moment[index_great_loss]

#output the test statistical information
visit_list=gaitdata_reader.get_visit_list(output_info,info_dic)
print('the size of planar pressure and moment data is seperately {} and {} \n'.format(valid_input[0].shape,valid_norm_moment[0].shape))
print(pd.value_counts(valid_task))


load all data from data buffer file

 ************************************************** 

该部分数据中共有 20 位患者，平均每位患者有3.45次回访，共计69次,平均每次回访测试了6.565217391304348趟，共计453趟

 

the size of planar pressure and moment data is seperately (100, 60, 63) and torch.Size([100, 18]) 

-24    78
-48    75
-36    74
-12    69
 36    48
 24    42
 48    34
 12    33
dtype: int64


## 2.data processing


In [12]:
#choose the task set and data argumentation 
task_set=[-12,-24,-36,-48]
this_input=[]
this_moment=[]
this_gaitdata=[]
this_delta=[]


print('The data of this trainning all from the task(week) of',task_set)
for i in range(len(valid_delta)):
    if valid_task[i] in task_set:
        e=footscan_reader.video_augment_padding(valid_input[i],70,70)
        this_input.extend(e)
        this_moment.extend([valid_norm_moment[i]]*5)
        this_delta.extend([valid_delta[i]]*5)
print('the trainning video numbe is {} after DATA ARGUMENTATION'.format(len(this_delta)))
print('input size after data argumentation is',this_input[0].shape)
print('*'*50)

The data of this trainning all from the task(week) of [-12, -24, -36, -48]
the trainning video numbe is 1480 after DATA ARGUMENTATION
input size after data argumentation is torch.Size([100, 70, 70])
**************************************************


In [ ]:
# choose one moment as our label
predict_category_index=3
print('we choose "{}” as our prediction label during this training'.format(this_gaitdata[1].category[predict_category_index]))
label_list=[]
for i in range(len(this_moment)):
    moment=this_moment[i]
    for j in range(moment.shape[0]):
        #3*i+1 means the y-axis
        label_list.append(moment[j,predict_category_index*3+1])

LABEL=torch.stack(label_list)
LABEL=torch.unsqueeze(LABEL,1)
label_ampli_factor=100
LABEL=LABEL*label_ampli_factor
print('we amplify the label {} times to get a mean value of {}'.format(label_ampli_factor,torch.mean(LABEL)))
print('the final label shape is',LABEL.shape)
print('*'*50)


In [ ]:
#get input from video——5 channels per frame
dif_amplify_factor=10
pressure_amlify_factor=10
input_list=[]
for i in range(len(this_input)):
    pressure_video=this_input[i]
    (t,h,w)=pressure_video.shape
    for j in range(t):
        this_pressure=pressure_video[j,:,:]
        if j==t-1:
            next_pressure=torch.zeros((h,w))
        else:
            next_pressure=pressure_video[j+1,:,:]
        
        if j==0:
            last_pressure=torch.zeros((h,w))
        else:
            last_pressure=pressure_video[j-1,:,:]
        #考虑进入时间因素（用delta来表示）
        dif_former=(this_pressure-last_pressure)/this_delta[i]*dif_amplify_factor
        dif_latter=(next_pressure-this_pressure)/this_delta[i]*dif_amplify_factor
        input_list.append(torch.stack([this_pressure,last_pressure,next_pressure,dif_former,dif_latter]))
DATA=torch.stack(input_list)
DATA=DATA*pressure_amlify_factor
print('DATA shape is',DATA.shape)
print('the mean abs value of the 5 channels of all data is: ',torch.mean(abs(DATA[0:5000,:,:,:]),dim=[0,2,3]))



In [ ]:
visit_list=gaitdata_reader.get_visit_list(this_gaitdata[0:50],info_dic)

In [ ]:
#extract the validation dataset--leave one subject out validation set
validation_index=[]
train_index=[]
leave_subject_name=this_visit[1][0]
for i in range(len(this_gaitdata)):
    if info_dic[this_gaitdata[i].name]==leave_subject_name:
        validation_index.extend(range(100*i,100*(i+1)))
    else:
        train_index.extend(range(100*i,100*(i+1)))
print('Use the leave-one-subject-out strategy to get the validation dataset.....')
print('the validation dataset is all data from {} consisting of {} pressure image input'.format(leave_subject_name,len(validation_index)))
print('the training set is from other participants consisting of {} pressure image input'.format(len(train_index)))

# for i in range(440):
#     if i%10==0:
#         validation_index.extend(range(100*i,100*(i+1)))
#     else:
#         train_index.extend(range(100*i,100*(i+1)))
# print('just use the validation evenly sampled to give a fake fitting example')


Final_dataset=[]
for i in range(LABEL.shape[0]):
    Final_dataset.append([DATA[i,:,:,:],LABEL[i,:]])
validation_sampler=SubsetRandomSampler(validation_index)
train_sampler=SubsetRandomSampler(train_index)
validation_loader=DataLoader(Final_dataset,BATCH_SIZE,sampler=validation_sampler,drop_last=True)
train_loader=DataLoader(Final_dataset,BATCH_SIZE,sampler=train_sampler,drop_last=True)
# num=0
# for x,y in test_loader:
#     print(x.shape,y.shape)
#     num+=1
# print(num)


## Build the moment-predicting netural networks

In [ ]:
#回归模型构建
'''
2023/4/22
处理足压片段来预测步态周期的力矩曲线

模型概况：
    金字塔conv
    flatten
    linear*1

输入输出：
    输入：该时刻的足压图像+该帧的变化量(batch,channel=5,height=180,width=63)
    输出：该时刻的膝关节力矩(batch)
'''

INPUT_FEATURES=5
HIDDEN_FEATURES_LIST=[16,32]
KERNEL_SIZE_LIST=[(6,6),(5,5),(7,7),(3,3)]
EPOCH_NUM=200

class Net(nn.Module):
    def __init__(self,batch_size,input_features,hidden_features_list,kernel_size_list):
        super(Net,self).__init__()

        self.batch_size = batch_size
        self.input_features = input_features
        self.hidden_features_list=hidden_features_list
        self.kernel_size_list=kernel_size_list

        self.BN1=nn.BatchNorm2d(input_features).to(DEVICE)
        self.pymarid_conv_1=nn.Conv2d(input_features,hidden_features_list[0],kernel_size_list[0],padding='same').to(DEVICE)                              
        self.pymarid_conv_2=nn.Conv2d(input_features,hidden_features_list[0],kernel_size_list[1],padding='same').to(DEVICE)
        self.pymarid_conv_3=nn.Conv2d(input_features,hidden_features_list[0],kernel_size_list[2],padding='same').to(DEVICE)
        self.conv2=nn.Sequential(
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.BatchNorm2d(hidden_features_list[0]*3),
            nn.Conv2d(hidden_features_list[0]*3,hidden_features_list[1],kernel_size_list[3],padding='same'),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.Linear=nn.Sequential(
            nn.Linear(hidden_features_list[1]*45*17,1600),
            nn.ELU(),
            nn.Linear(1600,512),
            nn.ELU(),
            nn.Linear(512,1),
        ).to(DEVICE)

    def forward(self,x):
        if x.ndim!=4:
            raise ValueError('输入数据ndim不为4')
        else:
            batch,features,height,width= x.shape
            if batch!=self.batch_size or features != self.input_features:
                raise ValueError('输入数据batch size 或者 input features 与model不符')
            else:
                pyramid_conv_output_list=[self.pymarid_conv_1(x),self.pymarid_conv_2(x),self.pymarid_conv_3(x)]
                pyramid_conv_output=torch.cat(pyramid_conv_output_list,dim=1)
                conv2_output=self.conv2(pyramid_conv_output)
                y=self.Linear(conv2_output.reshape(batch,-1))
                return y

model = Net(BATCH_SIZE,INPUT_FEATURES,HIDDEN_FEATURES_LIST,KERNEL_SIZE_LIST).to(DEVICE)     
print(model)
a = torch.ones((BATCH_SIZE,INPUT_FEATURES,180,70)).to(DEVICE)
b = model(a)
print(b.shape)

In [ ]:
# BEGIN TRAINNING
start = time.time()
def timesince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

#input shape:torch.size(batchsize,1)
def MRE(pred,train_y):
    relative_error=abs(pred-train_y)/train_y
    return torch.mean(relative_error)


def train(train_loader,test_loader,epoch_num,criterion,optimizer):
    train_loss=[]
    test_loss=[]
    train_NRMSE=[]
    test_NRMSE=[]
    train_MRE=[]
    test_MRE=[]
    early_stop_p = 0
    for i in range(epoch_num):
        #trainning process
        loss_each_batch=[]
        MRE_each_batch=[]
        NRMSE_each_batch=[]
        for x,y in train_loader:
            train_x=x.to(DEVICE)
            train_y=y.to(DEVICE)
            output=model(train_x).to(DEVICE)
            output=output.float()
            train_y=train_y.float()
            loss = criterion(output,train_y)
            if loss>10000:
                print(loss)
                print(j)
                print(output)
                print(train_y)
            optimizer.zero_grad()   
            loss.backward()
            optimizer.step()  

            loss_each_batch.append(loss.data.cpu())
            MRE_each_batch.append(MRE(output,train_y).data.cpu())
            NRMSE_each_batch.append(np.sqrt(loss.data.cpu())/(torch.max(train_y)-torch.min(train_y)).data.cpu())

        train_NRMSE.append(np.average(NRMSE_each_batch))
        train_loss.append(np.average(loss_each_batch))
        train_MRE.append(np.average(MRE_each_batch))

        #validation process
        with torch.no_grad():
            test_MRE_each_batch=[]
            test_loss_each_batch=[]
            test_NRMSE_each_batch=[]
            for test_x,test_y in test_loader:
                test_x=test_x.to(DEVICE)
                test_y=test_y.to(DEVICE)
                test_output=model(test_x)
                test_loss_item=criterion(test_output,test_y)
                test_loss_each_batch.append(test_loss_item.data.cpu())
                test_MRE_each_batch.append(MRE(output,train_y).data.cpu())
                test_NRMSE_each_batch.append(np.sqrt(test_loss_item.data.cpu())/(torch.max(test_y)-torch.min(test_y)).data.cpu())
            
            test_NRMSE.append(np.average(test_NRMSE_each_batch))
            test_loss.append(np.average(test_loss_each_batch))
            test_MRE.append(np.average(test_MRE_each_batch))


        print('Epoch [{}/{}],  Train loss(MSE): {:.4f}, TEST loss(MSE): {:.4f},time: {}'
                        .format(i, EPOCH_NUM, train_loss[-1],test_loss[-1],timesince(start)))
        
        #early_stop
        if i%3==0 and i>2:
            if test_loss[-1]>min(test_loss[max(0,i-5):i]):
                early_stop_p+=1
        if early_stop_p>4:
            break

    return train_loss,test_loss,train_NRMSE,test_NRMSE,train_MRE,test_MRE

CRITERION = nn.MSELoss(reduction='mean')
OPTIMIZER = torch.optim.Adam(model.parameters(), lr=LEARING_RATE)
train_loss,test_loss,train_NRMSE,test_NRMSE,train_MRE,test_MRE=train(train_loader,validation_loader,EPOCH_NUM,CRITERION,OPTIMIZER)

对数据进行单独点的测试，并进行可视化

In [ ]:
'''plot the predicting curve'''
def coefficient_deter(pred1,y1):
    error=(pred1-y1)*(pred1-y1)
    aver=np.average(y1)
    dis=(y1-aver)*(y1-aver)
    R2=1-np.sum(error)/np.sum(dis)
    return R2

DEVICE='cpu'
test_gait_loader=DataLoader(Final_dataset,BATCH_SIZE,sampler=validation_index)
train_gait_loader=DataLoader(Final_dataset,BATCH_SIZE,sampler=train_index)
model=model.cpu()
x_list=[]
ref_list=[]
pred_list=[]
coe_list=[]
iter_num=0
for x,y in test_gait_loader:
    pred_list.append(model(x).data.cpu().numpy())
    x_list.append(x.numpy())
    ref_list.append(y.numpy())
    coe_list.append(coefficient_deter(pred_list[-1],ref_list[-1]))
    iter_num+=1
    if iter_num>1:
        break
print(type(pred_list[0]))
plt.plot(pred_list[0][0:100,0],label='predicted moment')
plt.plot(ref_list[0][0:100],label='reference moment')
plt.xlabel('per centage of a gait')
plt.ylabel('knee moment')
plt.legend()
plt.show()
print('mean value of coefficient of determination of validation dataset is {}'.format(sum(coe_list)/len(coe_list)))

In [ ]:
plt.plot(test_loss,label='test loss')
plt.plot(train_loss,label='train loss')
plt.xlabel('epoch_num')
plt.ylabel('MSE(absolute value)')
plt.title('MSE_loss')
plt.legend()
plt.show()

# plt.plot(test_NRMSE,label='test_nrmse')
# plt.plot(train_NRMSE,label='train_nrmse')
# plt.xlabel('epoch_num')
# plt.ylabel('Noramlized root mean square error')
# plt.title('per cent')
# plt.legend()
# plt.show()

print('最后模型的NRMSE在训练集上是{},测试集上是{}'.format(train_NRMSE[-1],test_NRMSE[-1]))
print('最后模型的MRE在训练集上是{},测试集上是{}'.format(train_MRE[-1],test_MRE[-1]))


In [ ]:
plt.plot(test_MRE,label='test MRE')
plt.plot(train_MRE,label='train MRE')
plt.legend()
plt.ylim(-1,1)
plt.show()


# unusing code stroage

此时train 的 TRAIN_X和TRAIN_Y再做一次shuffle，因为之前是以一次测试的一百帧为单位进行打乱，所以处于一种宏观无序，但是微观有序的状态,这次打乱之后就微观无序了；
TRAIN_X_tensor_list=[]
TRAIN_Y_tensor_list=[]
origin_tensor_list=[]
for i in range(TRAIN_X_RAW.shape[0]):
    origin_tensor_list.append((TRAIN_X_RAW[i,:,:,:],TRAIN_Y_RAW[i,:]))
random.shuffle(origin_tensor_list)
for j in range(len(origin_tensor_list)):
    TRAIN_X_tensor_list.append(origin_tensor_list[j][0])
    TRAIN_Y_tensor_list.append(origin_tensor_list[j][1])
TRAIN_X=torch.stack(TRAIN_X_tensor_list)
TRAIN_Y=torch.stack(TRAIN_Y_tensor_list)
print(TRAIN_X.shape,TRAIN_Y.shape)




data shuffel
origin_list=[]
for i in range(len(this_input)):
    origin_list.append((this_input[i],this_moment[i],this_delta[i],this_gaitdata[i]))
random.shuffle(origin_list)
for i in range(len(origin_list)):
    this_input[i]=origin_list[i][0]
    this_moment[i]=origin_list[i][1]
    this_delta[i]=origin_list[i][2]
    this_gaitdata[i]=origin_list[i][3]
print('Finish the DATA SHUFFEL')
print('*'*50)
        